# Градиентный бустинг своими руками

**Внимание:** в тексте задания произошли изменения - поменялось число деревьев (теперь 50), правило изменения величины шага в задании 3 и добавился параметр `random_state` у решающего дерева. Правильные ответы не поменялись, но теперь их проще получить. Также исправлена опечатка в функции `gbm_predict`.

В этом задании будет использоваться датасет `boston` из `sklearn.datasets`. Оставьте последние 25% объектов для контроля качества, разделив `X` и `y` на `X_train`, `y_train` и `X_test`, `y_test`.

Целью задания будет реализовать простой вариант градиентного бустинга над регрессионными деревьями для случая квадратичной функции потерь.

In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np

import matplotlib
from matplotlib import pyplot as plt
matplotlib.style.use('ggplot')
%matplotlib inline

from sklearn import datasets

In [2]:
boston = datasets.load_boston()
X = boston.data
y = boston.target

In [3]:
from sklearn.model_selection import train_test_split

(X_train, 
 X_test, 
 y_train, y_test) = train_test_split(X, y, 
                                     test_size=0.25, 
                                     random_state=0)

## Задание 1

Как вы уже знаете из лекций, **бустинг** - это метод построения композиций базовых алгоритмов с помощью последовательного добавления к текущей композиции нового алгоритма с некоторым коэффициентом. 

Градиентный бустинг обучает каждый новый алгоритм так, чтобы он приближал антиградиент ошибки по ответам композиции на обучающей выборке. Аналогично минимизации функций методом градиентного спуска, в градиентном бустинге мы подправляем композицию, изменяя алгоритм в направлении антиградиента ошибки.

Воспользуйтесь формулой из лекций, задающей ответы на обучающей выборке, на которые нужно обучать новый алгоритм (фактически это лишь чуть более подробно расписанный градиент от ошибки), и получите частный ее случай, если функция потерь `L` - квадрат отклонения ответа композиции `a(x)` от правильного ответа `y` на данном `x`.

Если вы давно не считали производную самостоятельно, вам поможет таблица производных элементарных функций (которую несложно найти в интернете) и правило дифференцирования сложной функции. После дифференцирования квадрата у вас возникнет множитель 2 — т.к. нам все равно предстоит выбирать коэффициент, с которым будет добавлен новый базовый алгоритм, проигноируйте этот множитель при дальнейшем построении алгоритма.

In [9]:
def get_a_grad(algo):
    a_grad = []
    for pair in zip(X_train, y_train):
        a_grad.append(-2 * (algo(pair[0]) - pair[1]))
    return np.array(a_grad)

In [10]:
def kek(_):
    return 1

print(get_a_grad(kek))

[35.  37.2 64.4 24.2 13.  25.2 32.8 14.8 68.8 46.  24.8 50.4 12.4 24.2
 47.  72.4 48.  46.2 31.2 63.8 70.4 20.  12.4 43.6 55.4 26.8 46.8 34.2
 43.  39.  28.4 32.8 25.2 15.4 34.4 68.8 61.4 64.  42.4 38.8 45.8 48.
 23.4 56.2 22.  33.4 52.  39.2 18.4 33.  37.4 57.6 39.  27.8 19.8 37.
 43.4 37.  47.2 48.  47.  98.  26.6 21.6 60.  55.4 30.4 85.  48.  42.
 37.8 42.2 90.  43.8 38.4 84.2 67.2 25.6 46.6 41.  46.8 40.4 45.6 51.2
 48.2 17.2 36.8 36.8 17.  26.  51.  25.6 67.4 30.6 41.4 33.  29.2 39.8
 41.4 23.4 35.  45.4 36.6 23.4 41.2 44.4 57.2 40.4 45.6 32.2 42.  71.
 35.6 41.8 44.2 38.4 32.8 72.  46.2 70.4 29.4 62.4 25.  33.8 24.6 21.4
 81.4 34.8 24.2 48.  40.4 30.  67.8 48.4 47.6 41.  44.8 35.8 19.6 40.
 53.  33.  25.  55.4 27.6 36.2 55.2 24.2 36.  20.6 24.6 42.8 38.2 34.4
 43.8 39.2 48.  23.6 67.8 45.4 98.  56.  58.2 42.  29.2 44.6 58.2 26.6
 43.6 98.  39.6 10.6 67.8 62.8 37.8 38.6 33.6 44.2 38.8 44.4 12.  31.6
 91.4 98.  43.8 45.8 40.8 41.4 28.8 28.6 44.2 45.8 36.8 21.8 33.6 61.
 65.6 39.6 3

## Задание 2

Заведите массив для объектов `DecisionTreeRegressor` (будем их использовать в качестве базовых алгоритмов) и для вещественных чисел (это будут коэффициенты перед базовыми алгоритмами). 

В цикле от обучите последовательно 50 решающих деревьев с параметрами `max_depth=5` и `random_state=42` (остальные параметры - по умолчанию). В бустинге зачастую используются сотни и тысячи деревьев, но мы ограничимся 50, чтобы алгоритм работал быстрее, и его было проще отлаживать (т.к. цель задания разобраться, как работает метод). Каждое дерево должно обучаться на одном и том же множестве объектов, но ответы, которые учится прогнозировать дерево, будут меняться в соответствие с полученным в задании 1 правилом. 

Попробуйте для начала всегда брать коэффициент равным 0.9. Обычно оправдано выбирать коэффициент значительно меньшим - порядка 0.05 или 0.1, но т.к. в нашем учебном примере на стандартном датасете будет всего 50 деревьев, возьмем для начала шаг побольше.

В процессе реализации обучения вам потребуется функция, которая будет вычислять прогноз построенной на данный момент композиции деревьев на выборке `X`:

```
def gbm_predict(X):
    return [sum([coeff * algo.predict([x])[0] for algo, coeff in zip(base_algorithms_list, coefficients_list)]) for x in X]
(считаем, что base_algorithms_list - список с базовыми алгоритмами, coefficients_list - список с коэффициентами перед алгоритмами)
```

Эта же функция поможет вам получить прогноз на контрольной выборке и оценить качество работы вашего алгоритма с помощью `mean_squared_error` в `sklearn.metrics`. 

Возведите результат в степень 0.5, чтобы получить `RMSE`. Полученное значение `RMSE` — **ответ в пункте 2**.

In [20]:
def gbm_predict(X):
    return [sum([coeff * algo.predict([x])[0] for algo, 
                 coeff in zip(base_algorithms_list, coefficients_list)]) for x in X]

In [22]:
from sklearn.tree import DecisionTreeRegressor

base_algorithms_list = []
coefficients_list = [0.9] * 50


y_modified = y_train.copy()
for i in range(50):
    tmp_tree = DecisionTreeRegressor(max_depth=5, random_state=42).fit(X_train, y_modified)
    base_algorithms_list.append(tmp_tree)
    y_modified = y_train - gbm_predict(X_train)

In [23]:
from sklearn.metrics import mean_squared_error
gbm_predict(X_test)
mean_squared_error(y_test, gbm_predict(X_test)) ** 0.5

5.204116464919526

## Задание 3

Вас может также беспокоить, что двигаясь с постоянным шагом, вблизи минимума ошибки ответы на обучающей выборке меняются слишком резко, перескакивая через минимум. 

Попробуйте уменьшать вес перед каждым алгоритмом с каждой следующей итерацией по формуле `0.9 / (1.0 + i)`, где `i` - номер итерации (от 0 до 49). Используйте качество работы алгоритма как **ответ в пункте 3**. 

В реальности часто применяется следующая стратегия выбора шага: как только выбран алгоритм, подберем коэффициент перед ним численным методом оптимизации таким образом, чтобы отклонение от правильных ответов было минимальным. Мы не будем предлагать вам реализовать это для выполнения задания, но рекомендуем попробовать разобраться с такой стратегией и реализовать ее при случае для себя.

In [25]:
base_algorithms_list = []
coefficients_list = []


y_modified = y_train.copy()
for i in range(50):
    tmp_tree = DecisionTreeRegressor(max_depth=50, random_state=42).fit(X_train, y_modified)
    coefficients_list.append(0.9 / (1.0 + i))
    base_algorithms_list.append(tmp_tree)
    y_modified = y_train - gbm_predict(X_train)
    
gbm_predict(X_test)
mean_squared_error(y_test, gbm_predict(X_test)) ** 0.5

5.228002027704779

## Задание 4

Реализованный вами метод - градиентный бустинг над деревьями - очень популярен в машинном обучении. Он представлен как в самой библиотеке `sklearn`, так и в сторонней библиотеке `XGBoost`, которая имеет свой питоновский интерфейс. На практике `XGBoost` работает заметно лучше `GradientBoostingRegressor` из `sklearn`, но для этого задания вы можете использовать любую реализацию. 

Исследуйте, переобучается ли градиентный бустинг с ростом числа итераций (и подумайте, почему), а также с ростом глубины деревьев. На основе наблюдений выпишите через пробел номера правильных из приведенных ниже утверждений в порядке возрастания номера (это будет **ответ в п.4**):

    1. С увеличением числа деревьев, начиная с некоторого момента, качество работы градиентного бустинга не меняется существенно.

    2. С увеличением числа деревьев, начиная с некоторого момента, градиентный бустинг начинает переобучаться.

    3. С ростом глубины деревьев, начиная с некоторого момента, качество работы градиентного бустинга на тестовой выборке начинает ухудшаться.

    4. С ростом глубины деревьев, начиная с некоторого момента, качество работы градиентного бустинга перестает существенно изменяться

`2 3`

## Задание 5

Сравните получаемое с помощью градиентного бустинга качество с качеством работы линейной регрессии. 

Для этого обучите `LinearRegression` из `sklearn.linear_model` (с параметрами по умолчанию) на обучающей выборке и оцените для прогнозов полученного алгоритма на тестовой выборке `RMSE`. Полученное качество - ответ в **пункте 5**. 

В данном примере качество работы простой модели должно было оказаться хуже, но не стоит забывать, что так бывает не всегда. В заданиях к этому курсу вы еще встретите пример обратной ситуации.

In [30]:
from sklearn.linear_model import LinearRegression


model = LinearRegression().fit(X_train, y_train)
# model.predict(X_test)
mean_squared_error(y_test, model.predict(X_test))

29.782245092302375